In [ ]:
import pickle as pickle
import numpy as np
base_path="../data/data_m3/"
with open(base_path+'binary_train_codes_x.pkl', 'rb') as f0:
  binary_train_codes_x = pickle.load(f0)

with open(base_path+'binary_test_codes_x.pkl', 'rb') as f1:
  binary_test_codes_x = pickle.load(f1)

train_codes_y = np.load(base_path+'train_codes_y.npy')
train_visit_lens = np.load(base_path+'train_visit_lens.npy')

test_codes_y = np.load(base_path+'test_codes_y.npy')
test_visit_lens = np.load(base_path+'test_visit_lens.npy')
train_pids = np.load(base_path+'train_pids.npy')

test_pids = np.load(base_path+'test_pids.npy')
with open(base_path+'patient_time_duration_encoded.pkl', 'rb') as f80:
  patient_time_duration_encoded = pickle.load(f80)

In [ ]:
import numpy as np
import pickle

with open(base_path+"code_map.pkl", "rb") as f:
    code_map = pickle.load(f)

inv_code_map = {v-1: k for k, v in code_map.items()}
print(code_map)
def fun1(idx):
    result=[]
    for id in idx:
        result.append(inv_code_map[id])
    return result

In [ ]:
import pandas as pd
df = pd.read_csv(base_path+'diagnosis_icd9_ontology.csv')
code2name = dict(zip(df['code'], df['name']))

In [ ]:
def get_diag_str(diag_list):
    temp_str_list=[]
    for diag in diag_list:
        temp_str_list.append(f'"{code2name[diag]}"')
    temp_str=', '.join(temp_str_list)
    return temp_str

In [ ]:


def number_to_capitalized_ordinal(n):
    p = inflect.engine()
    return p.ordinal(p.number_to_words(n)).capitalize()

# 示例
print(number_to_capitalized_ordinal(1)) 
print(number_to_capitalized_ordinal(42)) 


In [ ]:

with open("pred_m3/m3_col_r1_result.pkl", "rb") as f:
    last_result = pickle.load(f)
len(last_result)

In [ ]:

import json
with open("cra_result_m3/m3_cra_r1.json", "r",encoding="utf-8") as f:
    hist_llm_result=json.load(f)

In [ ]:
import pandas as pd

df = pd.read_csv(base_path + 'diagnosis_icd9_ontology.csv')
code2name = dict(zip(df['code'], df['name']))
import pickle

with open(base_path + "code_map.pkl", "rb") as f:
    code_map = pickle.load(f)

inv_code_map = {v - 1: k for k, v in code_map.items()}

print(code_map['414.01'] - 1)
code_map_idx = list(code_map.keys())
import pandas as pd

df = pd.read_csv(base_path + 'diagnosis_icd9_ontology.csv')
df['code'] = df['code'].astype(str).str.strip()
df['name'] = df['name'].astype(str).str.strip()
df_f = df[df['code'].isin(code_map_idx)].copy()
name2code = dict(zip(df_f['name'].str.lower(), df_f['code']))


def extract_diagnoses_final(pred_str: str, name_to_code: dict):
    diagnoses_with_pos = []
    pred_str_lower = pred_str.lower()

    for name in name_to_code:
        name_lower = name.lower()
        index = pred_str_lower.find(name_lower)
        if index != -1:
            diagnoses_with_pos.append((index, name_lower))
    diagnoses_with_pos.sort(key=lambda x: x[0])

    return [name for _, name in diagnoses_with_pos]


def remove_duplicates_keep_order(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result


def get_diag_str(diag_list):
    temp_str_list = []
    for diag in diag_list:
        temp_str_list.append(f'"{code2name[diag]}"')
    temp_str = ', '.join(temp_str_list)
    return temp_str


import inflect


def number_to_capitalized_ordinal(n):
    p = inflect.engine()
    return p.ordinal(p.number_to_words(n)).capitalize()




In [ ]:
results=[]
ode_idx=0
top_k=50
for i in range(len(test_codes_y)):
    if test_visit_lens[i]<2:
        continue
    pids=test_pids[i]
    hist_diag=binary_test_codes_x[i]
    target_diag=test_codes_y[i]
    hist_diag_code=[]
    all_hist_diag_code=[]
    # ode_pred=data[ode_idx]
    # 
    # ode_idx+=1
    # ode_pred = np.argsort(-ode_pred, axis=-1)
    last=last_result[ode_idx]
    pred_str=hist_llm_result[ode_idx]['pred'].lower()
    ode_idx+=1
    for v in hist_diag:
        hist_diag_code.append(fun1(np.where(v == 1)[0]))
        all_hist_diag_code.extend(hist_diag_code[-1])
    # hist_diag_code.append(fun1(np.where(target_diag == 1)[0]))
    all_hist_diag_code=list(set(all_hist_diag_code))
    
    instruction='''
You are an experienced medical diagnosis expert specializing in longitudinal disease progression and comorbidity reasoning.

Your task is to UPDATE the diagnosis list for the patient’s final visit by carefully comparing:
(1) the Previous Agent Output (CRA),
and
(2) the Collaborative Multi-Expert Consensus (COL),
together with the patient’s longitudinal diagnosis history.
    '''

    
    duration=patient_time_duration_encoded[pids]
    
    input_=""
    for idx,code_list in enumerate(hist_diag_code):
        if idx==0:
            cur_duration=0
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit:"
        else:
            cur_duration+=duration[idx]
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit: ({cur_duration} days later):"

        temp+=f"\n- Diagnoses："+"{"+get_diag_str(code_list)+"}"
        input_+=temp+"\n\n"
    temp=f"Final Visit ({sum(duration)} days later):\n\n"

    pred_str=pred_str.replace("*","")
    diagnoses=extract_diagnoses_final(pred_str.lower(),name2code)


    codes = []
    for name in diagnoses:
        code = name2code.get(name)
        if code is not None:
            codes.append(code)
    
    pred_list=codes
    pred_list = remove_duplicates_keep_order(pred_list)
    temp_pred=[code_map[code]-1 for code in pred_list]
    temp+='''Previous Agent Output: \n{'''+get_diag_str(fun1(temp_pred))+"}"
    temp+='''\n\nCollaborative Multi-Expert Consensus: \n{'''+get_diag_str(fun1(last))+"}"
    temp+='''   
\n\nYour task:
- Update the final-visit diagnosis list by integrating the longitudinal disease trajectory, recurrence patterns, and complication chains.
- When revising the list, prioritize progressive, persistent, or downstream secondary conditions supported by the longitudinal history, and systematically down-rank transient, reversible, or treatment-related acute events unlikely to persist.
- Carefully compare CRA and COL to refine the output: retain clinically plausible and persistent conditions agreed upon or supported by the trajectory, remove implausible or nonpersistent items, and supplement missing chronic/progressive diseases highlighted by COL when they are justified by longitudinal evidence.

Directly provide the reordered list of disease names in descending order of likelihood. \nOutput format:\nAnswer: <Disease 1>, <Disease 2>, <Disease 3>, ...
    '''
    input_+=temp+"\n"
    result={
        "pid":pids,
        "hist_diag_code":hist_diag_code,
        "target_diag_code":fun1(np.where(target_diag == 1)[0]),
        "instruction":instruction,
        "input":input_
        
    }

    results.append(result)


In [ ]:
import json
def np_encoder(obj):
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()          # 转成 Python int/float/bool
    if isinstance(obj, np.ndarray):
        return obj.tolist()        # 转成原生 list
    return str(obj)                # 兜底（可选）

with open(f"cra_prompt_m3/m3_cra_r2.json", "w",
          encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False,default=np_encoder, indent=2)